## Introduction

In [1]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

'Connected: None@factbook.db'

## Overview of the Data

In [2]:
%%sql
SELECT *
  FROM sqlite_master;


Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


In [25]:
%%sql

SELECT *
FROM  facts
LIMIT 5

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


Here are the descriptions for some of the columns:

* **name** - The name of the country.
* **area** - The total land and sea area of the country.
* **population** - The country's population.
* **population_growth** - The country's population growth as a percentage.
* **birth_rate** - The country's birth rate, or the number of births a year per 1,000 people.
* **death_rate** - The country's death rate, or the number of death a year per 1,000 people.
* **area** - The country's total area (both land and water).
* **area_land** - The country's land area in square kilometers.
* **area_water** - The country's waterarea in square kilometers.

Let's start by calculating some summary statistics and see what they tell us.

In [4]:
%%sql
SELECT MIN(population) as 'Minimum_population',
       MAX(population) as 'Maximum_population',
       MIN(population_growth) as 'Min_popul_growth',
       MAX(population_growth) as 'Max_popul_growth'
FROM facts

Done.


Minimum_population,Maximum_population,Min_popul_growth,Max_popul_growth
0,7256490011,0.0,4.02


A few things stick out from the summary statistics in the last screen:

* There's a country with a population of 0
* There's a country with a population of 7256490011 (or more than 7.2 billion people)

Let's use subqueries to zoom in on just these countries 

## Exploring Outliers

In [5]:
%%sql
SELECT *
FROM facts
WHERE population = (SELECT MAX(population)
                   FROM facts)
   OR population = (SELECT MIN(population)
                   FROM facts)

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
250,ay,Antarctica,None,280000,None,0,None,None,None,None
261,xx,World,None,None,None,7256490011,1.08,18.6,7.8,None


## Summary Statistics Revisited

Now that we know more about the data, we should recalculate the summary statistics we calculated earlier, while excluding the row for the whole world.

In [6]:
%%sql
SELECT MIN(population) as 'Minimum_population',
       MAX(population) as 'Maximum_population',
       MIN(population_growth) as 'Min_popul_growth',
       MAX(population_growth) as 'Max_popul_growth'
FROM facts
WHERE name NOT IN ('World', 'Antarctica')

Done.


Minimum_population,Maximum_population,Min_popul_growth,Max_popul_growth
48,1367485388,0.0,4.02


## Exploring Average Population and Area

Let's explore density. Density depends on the population and the country's area. Let's look at the average values for these two columns.

We should take care of discarding the row for the whole planet.

In [12]:
%%sql
SELECT CAST(AVG(population) AS Int) AS Avg_popul, CAST(AVG(area) AS Int) as Avg_area
FROM facts
WHERE name NOT IN ('World', 'Antarctica')

Done.


Avg_popul,Avg_area
32377011,555093


## Finding Densely Populated Countries

In [16]:
%%sql
SELECT *
FROM facts
WHERE population > (SELECT AVG(population) 
                    FROM facts
                    WHERE name <> 'World')
        AND
      area_land < (SELECT AVG(area_land)
                   FROM facts
                   WHERE name <> 'World')
ORDER BY population DESC

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
14,bg,Bangladesh,148460,130170,18290,168957745,1.6,21.14,5.61,0.46
85,ja,Japan,377915,364485,13430,126919659,0.16,7.93,9.51,0.0
138,rp,Philippines,300000,298170,1830,100998376,1.61,24.27,6.11,2.09
192,vm,Vietnam,331210,310070,21140,94348835,0.97,15.96,5.93,0.3
65,gm,Germany,357022,348672,8350,80854408,0.17,8.47,11.42,1.24
173,th,Thailand,513120,510890,2230,67976405,0.34,11.19,7.8,0.0
185,uk,United Kingdom,243610,241930,1680,64088222,0.54,12.17,9.35,2.54
83,it,Italy,301340,294140,7200,61855120,0.27,8.74,10.19,4.1
91,ks,"Korea, South",99720,96920,2800,49115196,0.14,8.19,6.75,0.0
163,sp,Spain,505370,498980,6390,48146134,0.89,9.64,9.04,8.31


## Additional questions to look into

* Which country has the most people? 

In [18]:
%%sql
SELECT name, population
FROM facts
WHERE population = (SELECT MAX(population) 
                    FROM facts 
                    WHERE name <> 'World')

Done.


name,population
China,1367485388


* Which country has the highest growth rate?

In [19]:
%%sql
SELECT name, population_growth
FROM facts
WHERE population_growth = (SELECT MAX(population_growth)
                          FROM facts
                          WHERE name <> 'World')

Done.


name,population_growth
South Sudan,4.02


* Which countries have the highest ratios of water to land?  Which countries have more water than land?

In [27]:
%%sql
SELECT name, CAST(area_water as Float)/CAST(area_land as Float) as ratio_water_to_land
FROM facts
ORDER BY ratio_water_to_land DESC
LIMIT 10


Done.


name,ratio_water_to_land
British Indian Ocean Territory,905.6666666666666
Virgin Islands,4.520231213872832
Puerto Rico,0.5547914317925592
"Bahamas, The",0.3866133866133866
Guinea-Bissau,0.2846728307254623
Malawi,0.25939625850340137
Netherlands,0.22571032366565366
Uganda,0.22292237442922375
Eritrea,0.16435643564356436
Liberia,0.15623961794019933


* Which countries will add the most people to their populations next year?

In [28]:
%%sql
SELECT name, population_growth
FROM facts
ORDER BY population_growth DESC
LIMIT 10

Done.


name,population_growth
South Sudan,4.02
Malawi,3.32
Burundi,3.28
Niger,3.25
Uganda,3.24
Qatar,3.07
Burkina Faso,3.03
Mali,2.98
Cook Islands,2.95
Iraq,2.93


* Which countries have a higher death rate than birth rate?

In [30]:
%%sql
SELECT name, death_rate, birth_rate
FROM facts
WHERE death_rate>birth_rate
ORDER BY death_rate DESC

Done.


name,death_rate,birth_rate
Ukraine,14.46,10.72
Bulgaria,14.44,8.92
Latvia,14.31,10.0
Lithuania,14.27,10.1
Russia,13.69,11.6
Serbia,13.66,9.08
Belarus,13.36,10.7
Hungary,12.73,9.16
Moldova,12.59,12.0
Estonia,12.4,10.51


* Which countries have the highest `population/area` ratio?

In [33]:
%%sql
SELECT name, population/area_land AS ratio_popul_land
FROM facts
ORDER BY ratio_popul_land DESC
limit 10

Done.


name,ratio_popul_land
Macau,21168
Monaco,15267
Singapore,8259
Hong Kong,6655
Gaza Strip,5191
Gibraltar,4876
Bahrain,1771
Maldives,1319
Malta,1310
Bermuda,1299
